Import Libraries required

In [1]:
import pandas
import numpy

Continue from the Assignment 2 Data

In [2]:
file1 = pandas.read_csv('Assignment-2-CleanedData.csv')

Drop Unwanted column

In [3]:
file1.drop('Unnamed: 0', axis=1, inplace = True)


Import the required frameworks

In [4]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas_datareader as pdr
import datetime as dt

Get data from JSON for ticker symbols

In [6]:
import requests
import json
import urllib

def get_TickerSymbol(companyName):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(companyName)

    result = requests.get(url).json()
    
    for x in result['ResultSet']['Result']:
                    return x['symbol']
        

arrTicker=[]
comp_name = file1.Entity

    
for  name in comp_name:
    arrTicker.append(get_TickerSymbol(name))
    
    


Collect share data between two dates for companies with ticker symbols

In [10]:
start = dt.datetime(2004,1,1)
end = dt.datetime(2017,6,6)
panel_data = pdr.DataReader(arrTicker, 'yahoo', start, end)

/Users/agasthya/anaconda3/lib/python3.6/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: 'AOL', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/agasthya/anaconda3/lib/python3.6/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: None, replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/agasthya/anaconda3/lib/python3.6/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: 'CIT-U.TI', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/agasthya/anaconda3/lib/python3.6/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: '^UMIAMIFL', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/agasthya/anaconda3/lib/python3.6/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: 'JPM-PB', replacing with NaN.
  warnings

In [11]:
panel_data

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 3799 (major_axis) x 61 (minor_axis)
Items axis: Adj Close to Volume
Major_axis axis: 2004-01-01 00:00:00 to 2017-06-07 00:00:00
Minor_axis axis: ADP to FLINGX

In [13]:
# The index in this DataFrame is the major index of the panel_data.
finalData = panel_data.ix['Close']

# Getting all weekdays between 01/01/2000 and 12/31/2016
all_weekdays = pandas.date_range(start=start, end=end, freq='B')

# How do we align the existing prices in adj_close with our new set of dates?
# All we need to do is reindex close using all_weekdays as the new index
finalData = finalData.reindex(all_weekdays)


In [14]:
finalData2 = finalData.T

In [15]:
finalData2 = finalData.groupby(pandas.TimeGrouper(freq='12M')).mean()

In [16]:
finalData2 = finalData2.T


In [17]:

newDataFrame= pandas.DataFrame(columns=['Symbol','Date','Price'])
k =0
for i in range(len(finalData2)) : 
    ts = finalData2.index[i]
    for j in range(len(finalData2.iloc[i])): 
        time_frame = pandas.to_datetime(str(finalData2.columns.values[j]))
        date = time_frame.strftime('%Y-%m-%d')
        newDataFrame.loc[k]=( ts,  date , finalData2.iloc[i][j])
        k=k+1
        
        

In [18]:
newDataFrame.head()

,Symbol,Date,Price
0,ADP,2004-01-31,32.813820
1,ADP,2005-01-31,34.064935
2,ADP,2006-01-31,35.050695
3,ADP,2007-01-31,36.863325
4,ADP,2008-01-31,40.408635


Change the year format

In [19]:
newDataFrame.loc[newDataFrame['Date'] == '2005-01-31', 'Date'] = '2004'
newDataFrame.loc[newDataFrame['Date'] == '2006-01-31', 'Date'] = '2005'
newDataFrame.loc[newDataFrame['Date'] == '2007-01-31', 'Date'] = '2006'
newDataFrame.loc[newDataFrame['Date'] == '2008-01-31', 'Date'] = '2007'
newDataFrame.loc[newDataFrame['Date'] == '2009-01-31', 'Date'] = '2008'
newDataFrame.loc[newDataFrame['Date'] == '2010-01-31', 'Date'] = '2009'
newDataFrame.loc[newDataFrame['Date'] == '2011-01-31', 'Date'] = '2010'
newDataFrame.loc[newDataFrame['Date'] == '2012-01-31', 'Date'] = '2011'
newDataFrame.loc[newDataFrame['Date'] == '2013-01-31', 'Date'] = '2012'
newDataFrame.loc[newDataFrame['Date'] == '2014-01-31', 'Date'] = '2013'
newDataFrame.loc[newDataFrame['Date'] == '2015-01-31', 'Date'] = '2014'
newDataFrame.loc[newDataFrame['Date'] == '2016-01-31', 'Date'] = '2015'
newDataFrame.loc[newDataFrame['Date'] == '2017-01-31', 'Date'] = '2016'
newDataFrame.loc[newDataFrame['Date'] == '2018-01-31', 'Date'] = '2017'


In [ ]:
#Change the Date to year before. ignoring 2004. 

In [20]:
newDataFrame = newDataFrame[newDataFrame['Date'] != '2004-01-31']

In [21]:
#Save Data to a file
newDataFrame.to_csv('Assignment3_FinanceFinalData.csv', index=True, encoding='utf-8')


In [22]:
data_ticker = pandas.DataFrame(arrTicker) 
data_ticker['Ticker_Symbol'] = pandas.Series(arrTicker, index=data_ticker.index)

In [23]:
file1['Ticker_symbol']= data_ticker['Ticker_Symbol']


In [24]:
file1 = file1.dropna(axis=0,how='any')


In [25]:
file1.head()

,Entity,Year,Records Lost,Organisation,Method of Leak,Data Sensitivity,Ticker_symbol
0,AOL,2004,92000000,web,inside job,E-mail Address or Online Information,AOL
1,Automatic Data Processing,2005,125000,financial,poor security,SSN/Personal Details,ADP
3,Citigroup,2005,3900000,financial,lost / stolen device,Credit Card Information,CIT-U.TI
5,Hewlett Packard,2006,200000,"tech, retail",lost / stolen device,SSN/Personal Details,HPQ
7,KDDI,2006,4000000,telecoms,hacked,E-mail Address or Online Information,KDDIF


In [26]:
file1.to_csv('Assignment7_FinanceData.csv', index=False, encoding='utf-8')


Original Claim -
'Data breaches are not punished by stock market' as long as the company is well received or followed by customers. Facebook is the classic example, even when there was a data breach, the stock price of Facebook never really came down and was always increasing because of its popularity and ever increasing user base.
The dashboard shows the graph of stock price of Facebook.

Purpose of Redesign - 
The purpose of this redesign is to take a look at the old visualization and understand if there needs to be a change in the way the previous version has been represented. 

Original vs Redesign

The claim from my previous visualization is that ‘Data breaches are not punished by stock markets. The redesign of the earlier visualization also supports the same claim, the visualization is also similar to the previous one and has very little changes. I feel that this visualization has just about enough data that supports my claim but I have now excluded the part of the graph which gives information about all the data breaches, the type of breach and number of records lost during those breaches. I feel this information in a final presentation of the data in unnecessary because it doesn’t provide any required additional data to support my claim for the visualization and also it might not be the best way to present my data to the users and make them read more. The graph which shows that Facebook’s stock price was not affected by the data breach, the stock price kept on increasing, this graph is sufficient to support my claim. I have also converted this graph to a simple line chart. The final graph gives an idea of how Facebook's share price were not affected by the data breach that had occured. The share price kept increasing steadily due to the companies customer base and popularity.


If I had more data on the exact dates of the breaches, the dates that the companies announced about these breaches, that would be a better case as it would be possible to look at multiple companies who might have lost their stock price due to these breaches and not because of other market fluctuations such as Citi bank which lost its share price due the market meltdown in 2008-09.

Link to Tableau Public for Original - 
https://public.tableau.com/profile/aashreya.agasthya.kundurthy#!/vizhome/assignment3_1_1/Dashboard1?publish=yes

Link to Tableau Public for Redesign -
https://public.tableau.com/profile/aashreya.agasthya.kundurthy#!/vizhome/Assignment-7_0/Redesigned-Dashboard?publish=yes

Reference -
https://docs.google.com/spreadsheets/d/1Je-YUdnhjQJO_13r8iTeRxpU2pBKuV6RVRHoYCgiMfg/edit#gid=322165570




